# 2020 SUMMER ESC :: Week 3 세션 과제

2020.08.09 김채형

### 과제 1번

1. 이미지 파일에 zero padding을 크기 1만큼 적용한 뒤 (즉 위 아래 양옆으로 0을 한겹만)
2. stride를 (1,1)로 filter A를 적용하고
3. stride (2,2)로 2\*2 maxpooling을 적용했을 때의 이미지 파일은 어떻게 생겼는지 예측해주세요!

In [20]:
mat = [[2,0,2,1,3,2],
       [0,2,0,2,2,2],
       [1,0,1,3,1,1],
       [0,0,1,1,1,0],
       [0,1,3,4,1,0],
       [0,1,0,0,5,2]]
mat

[[2, 0, 2, 1, 3, 2],
 [0, 2, 0, 2, 2, 2],
 [1, 0, 1, 3, 1, 1],
 [0, 0, 1, 1, 1, 0],
 [0, 1, 3, 4, 1, 0],
 [0, 1, 0, 0, 5, 2]]

In [21]:
fil = [[1,0,0],
       [0,1,0],
       [0,0,1]]
fil

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [22]:
# zero padding 크기 1만큼 적용
mat_padded = []
padded_len = len(mat)+2

for i in range(padded_len):
    mat_padded.append([0]*padded_len)
    #print(mat_padded)
    if i!=0 and i!=padded_len-1:
        mat_padded[i][1:-1] = mat[i-1]
        #print(mat_padded, '\n')

In [23]:
# zero padding 적용 후 이미지
mat_padded

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 2, 1, 3, 2, 0],
 [0, 0, 2, 0, 2, 2, 2, 0],
 [0, 1, 0, 1, 3, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 1, 3, 4, 1, 0, 0],
 [0, 0, 1, 0, 0, 5, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [24]:
# filter A를 stride를 (1,1)로 적용
mat_filtered = []
for i in range(len(mat)):
    ls = []
    for j in range(len(mat)):
        val = mat_padded[i][j] + mat_padded[i+1][j+1] + mat_padded[i+2][j+2]
        ls.append(val)
    mat_filtered.append(ls)

In [25]:
# filter A 적용 후 이미지
mat_filtered

[[4, 0, 4, 3, 5, 2],
 [0, 5, 3, 5, 4, 5],
 [1, 1, 4, 4, 3, 3],
 [1, 4, 5, 3, 4, 1],
 [1, 1, 3, 10, 4, 1],
 [0, 1, 1, 3, 9, 3]]

In [26]:
# 2*2 maxpooling을 stride (2,2)로 적용
mat_maxpooled = []
for i in range(3):
    ls = []
    for j in range(3):
        val = max(mat_filtered[2*i][2*j], mat_filtered[2*i+1][2*j], mat_filtered[2*i][2*j+1], mat_filtered[2*i+1][2*j+1])
        ls.append(val)
    mat_maxpooled.append(ls)

In [27]:
# maxpooling 적용 후 이미지
mat_maxpooled

[[5, 5, 5], [4, 5, 4], [1, 10, 9]]

### 과제 2번

1. Layer1의 Kernel size를 4로 늘리고, Layer3의 stride를 2로 늘려라
2. Layer3의 Output size를 구하여라
3. Cost와 Accuracy가 기존 CNN 모델에 비해 어떻게 변할 것인가?
4. 코딩하여 Cost와 Accuracy를 확인해보자!

In [48]:
torch.manual_seed(777)

In [49]:
# 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [50]:
# 데이터셋 불러오기
mnist_train = dsets.MNIST(root='MNIST_data',
                          train=True, 
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data',
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

In [51]:
# data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [53]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        self.fc1 = nn.Linear(128*2*2, 625, bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = nn.Sequential(
            self.fc1, 
            nn.ReLU(),
            nn.Dropout(p=0.3)
        )
        self.fc2 = nn.Linear(625, 10, bias=True)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x.float())
        out = self.layer2(out.float())
        out = self.layer3(out.float())
        out = out.view(out.size(0), -1) # Flatten before FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [54]:
model = CNN()

In [55]:
# cost & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [56]:
# training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))
    
print('Learning Finished!')

[Epoch:    1] cost = 0.243409455
[Epoch:    2] cost = 0.0625862181
[Epoch:    3] cost = 0.0428103581
[Epoch:    4] cost = 0.0314080901
[Epoch:    5] cost = 0.0271734335
[Epoch:    6] cost = 0.0211458337
[Epoch:    7] cost = 0.0187178925
[Epoch:    8] cost = 0.0161897969
[Epoch:    9] cost = 0.013910831
[Epoch:   10] cost = 0.0132223396
[Epoch:   11] cost = 0.00972700864
[Epoch:   12] cost = 0.00994654559
[Epoch:   13] cost = 0.0107850786
[Epoch:   14] cost = 0.00808450952
[Epoch:   15] cost = 0.0101576112
Learning Finished!


In [58]:
# accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28)
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/kimchaehyeong/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/kimchaehyeong/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9848999977111816


기존 모델 : cost = 0.0030285837 / accuracy = 0.988099992275238  
바꾼 모델 : cost = 0.0101576112 / accuracy = 0.9848999977111816